In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w4p2/sample.csv
/kaggle/input/2022-ai-w4p2/train.csv
/kaggle/input/2022-ai-w4p2/test.csv


In [2]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('../input/2022-ai-w4p2/train.csv')
test = pd.read_csv('../input/2022-ai-w4p2/test.csv')
submit = pd.read_csv('../input/2022-ai-w4p2/sample.csv')

In [4]:
trainX = train.drop('label', axis=1)
trainY = train['label']

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainY = le.fit_transform(trainY)

In [6]:
trainY

array([13, 19, 17, ..., 12,  7, 10])

In [7]:
x_train = torch.FloatTensor(trainX.to_numpy())
y_train = torch.LongTensor(trainY)
x_test = torch.FloatTensor(test.to_numpy())

In [8]:
y_train.shape

torch.Size([1650])

In [9]:
## 데이터 학습

# softmax
import torch.nn.functional as F

nb_data = x_train.shape[1]
nb_class = 22

W = torch.zeros([nb_data, nb_class], requires_grad=True)
b = torch.zeros(nb_class, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.01)

nb_epochs = 10000
for epoch in range(nb_epochs+1) :
    
    hypothesis = F.softmax(torch.matmul(x_train, W)+b, dim=1)
    
    # cross_entropy를 사용하면 scatter 함수를 이용한 one_hot_encoding을 안해도 됨
    cost = F.cross_entropy((x_train.matmul(W) + b), y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 1000 == 0 :
        print(epoch, cost.item())

0 3.0910415649414062
1000 0.5660277009010315
2000 0.4325185716152191
3000 0.1499970555305481
4000 0.12525497376918793
5000 0.1457214206457138
6000 0.1420230269432068
7000 0.13809983432292938
8000 0.1346563696861267
9000 0.13151179254055023
10000 0.12853045761585236


In [10]:
hypothesis = F.softmax(torch.matmul(x_test, W)+b, dim=1)
predict = torch.argmax(hypothesis, dim=1)

In [11]:
submit['label'] = le.inverse_transform(predict)

In [12]:
submit.to_csv('submit.csv', index=False)